# Single Path One-Shot Neural Architecture Search using Random Search

# Search the best architecture

## <span style="color:red">In previous notebooks, we actually missed one important fact...</span>

Let's talk about **Batch Normalization**. Yes, they are always pain in the ...

So, the problem is the following.

When we're training the supernet, how do batch norm layers behave? They're computing **per-batch** mean and variance for each sampled architecture. This actually helps SPOS **a lot**. We can easily imagine that operations with different number of intermediate channels (from 0.25 to 1.5) can have very different output distributions. Well, per-batch statistics can handle this easily, **each block just normalizes the output of previously sampled operation**, and it is good to go!

As you might know, the behaviour of batch norm layers is different in eval mode. It uses accumulated statistics for each batch norm layer. But what is accumulated there?

The results are corrupted. The running stats are averaged using various architectures during training, which is not good.

The solution for this is to **tune batch norm statistics** for each sampled architecture. So, overall our model evaluation pipeline would look like this:

1. Sample random architecture;
2. Move it to `train` mode;
3. Tune it's statistics on some number of training batches;
4. Then switch to `eval` mode;
5. Evaluate.

Note that we're using training data for batch norm tuning to prevent overfitting.

## Setup

### Imports

In [1]:
import json
import os
import sys
from typing import Dict

import numpy as np
import torch
import torchvision
from thop import profile
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm

from cifar10 import get_train_transform, get_val_transform
from supernet import supernet18

### Make everything a bit faster

In [2]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

### Build datasets and dataloaders for CIFAR-10

In [3]:
# Change this value if needed.
batch_size = 512

In [4]:
train_transform = get_train_transform()
val_transform = get_val_transform()

train_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=train_transform,
)
test_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=val_transform,
)

train_dataloader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    drop_last=True,
)
test_dataloader = DataLoader(
    test_set,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    drop_last=False,
)

Files already downloaded and verified
Files already downloaded and verified


## Load trained supernet

In [5]:
# Select suitable device.
# You should probably use either cuda (NVidia GPU) or mps (Apple) backend.
device = torch.device('cuda:0')

In [6]:
channel_multipliers = (0.25, 0.5, 0.75, 1.0, 1.25, 1.5)
supernet = supernet18(num_classes=10, zero_init_residual=True, channel_multipliers=channel_multipliers)
supernet.to(device=device)

Supernet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): SearchBlock(
      (ops): ModuleList(
        (0): BasicBlock(
          (conv1): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(32, eps=1e-05,

In [7]:
checkpoint = torch.load('supernet_advanced_360.pth', map_location=device)
supernet.load_state_dict(checkpoint)

<All keys matched successfully>

## Re-evaluate models from previous random search

### Load info from previous random search

In [8]:
with open('random_search_1000_models_supernet_advanced_360.json', 'r') as f:
    search_data = json.load(f)

### Evaluate losses and accuracies with tuned batch norms

In [9]:
criterion = nn.CrossEntropyLoss()

In [10]:
def clean_running_stats(layer: nn.Module) -> None:
    if isinstance(layer, nn.BatchNorm2d):
        layer.momentum = None
        layer.reset_running_stats()

In [11]:
search_data['tuned_accuracy'] = []
search_data['tuned_loss'] = []

In [12]:
# Select the number of batches for batch norm tuning.
# This value is arbitrary, it seems like 5120 samples is enough
# to collect reasonable statistics.
n_batches_for_tuning = 10

In [13]:
model = supernet

total_architectures = len(search_data['arch'])
zip_arch_accuracy_loss = zip(search_data['arch'], search_data['accuracy'], search_data['loss'])
arch_iter = tqdm(enumerate(zip_arch_accuracy_loss), total=total_architectures)

for i, (arch, old_accuracy, old_loss) in arch_iter:
    model.sample(arch)

    # EACH evaluation, we apply `clean_running_stats` function to all batch norm layers.
    # It does two things:
    #
    # 1. Sets momentum to None. This uses plain statistics averaging instead of relying
    #    on running estimations.
    # 2. Resets running stats for each batch norm layer. This ensures that each time we
    #    average batch norm statistics only using one architecture.
    #
    # For further understanding, read PyTorch docs and sources, or read original article
    # about batch norms:
    #   https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html
    #   https://pytorch.org/docs/stable/_modules/torch/nn/modules/batchnorm.html#BatchNorm2d
    #   https://arxiv.org/abs/1502.03167
    model.apply(clean_running_stats)

    # Move model to `train` mode to update batch norm statistics.
    model.train()

    # Run few batches to collect statistics.
    for i, (train_inputs, _) in enumerate(train_dataloader):
        with torch.no_grad():
            model(train_inputs.to(device=device))

        # Do not use all training data for batch norm tuning.
        if i == n_batches_for_tuning - 1:
            break

    # Move model to `eval` mode to use accumulated statistics.
    model.eval()

    total_loss = 0.0
    total_correct = 0.0
    total_samples = 0
    for inputs, labels in test_dataloader:
        inputs = inputs.to(device=device)
        labels = labels.to(device=device)

        logits = model(inputs)
        loss = criterion(logits, labels) * labels.shape[0]
        _, predicted_labels = torch.max(logits, 1)
        total_loss += loss.item()
        total_correct += (predicted_labels == labels).sum().item()
        total_samples += labels.shape[0]

    accuracy = total_correct / total_samples
    loss = total_loss / total_samples
    search_data['tuned_accuracy'].append(accuracy)
    search_data['tuned_loss'].append(loss)
    print(
        f'{arch}: '
        f'accuracy={old_accuracy:.2%} -> {accuracy:.2%}, '
        f'loss={old_loss:.4f} -> {loss:.4f}'
    )

  0%|▏                                                                                                                                    | 1/1000 [00:01<28:57,  1.74s/it]

[0, 3, 4, 3, 5, 3, 5, 5]: accuracy=83.79% -> 84.33%, loss=0.4735 -> 0.4672


  0%|▎                                                                                                                                    | 2/1000 [00:02<21:37,  1.30s/it]

[0, 1, 1, 3, 0, 0, 3, 3]: accuracy=82.50% -> 83.23%, loss=0.5159 -> 0.5088


  0%|▍                                                                                                                                    | 3/1000 [00:03<19:13,  1.16s/it]

...

[0, 0, 0, 3, 2, 4, 4, 2]: accuracy=80.77% -> 81.73%, loss=0.5634 -> 0.5447


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 999/1000 [15:56<00:00,  1.04it/s]

[3, 4, 3, 1, 5, 2, 5, 0]: accuracy=85.02% -> 85.13%, loss=0.4518 -> 0.4370


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [15:57<00:00,  1.04it/s]

[0, 2, 2, 0, 2, 3, 3, 1]: accuracy=82.68% -> 83.53%, loss=0.5036 -> 0.4863


### <span style="color:green">Note that accuracies improved in almost all the cases, thus incurring more fair evaluation of architectures. And test losses... all of them improved.</span>

### Save updated results for further analysis

In [14]:
with open('random_search_1000_models_tune_bn_supernet_advanced_360.json', 'w') as f:
    json.dump(search_data, f)